In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install langchain langchain-nvidia-ai-endpoints pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plot

In [3]:
from pydantic import BaseModel, Field
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [4]:
from kaggle_secrets import UserSecretsClient
nvidia_api_key = UserSecretsClient().get_secret("NVIDIA_API_KEY")
nvidia_model_name = UserSecretsClient().get_secret("NVIDIA_MODEL_NAME")

In [5]:
llm = ChatNVIDIA(
    model = nvidia_model_name,
    api_key = nvidia_api_key,
)

In [6]:
class Country(BaseModel):
    """ Information about a country """
    name: str = Field(description = "name of the country")
    language: str = Field(description = "language of the country")
    capital: str = Field(description = "Capital of the country" )
    

In [7]:
structured_llm = llm.with_structured_output(Country)
structured_llm

RunnableBinding(bound=ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='meta/llama-3.1-70b-instruct'), kwargs={'nvext': {'guided_json': {'description': 'Information about a country ', 'properties': {'name': {'description': 'name of the country', 'title': 'Name', 'type': 'string'}, 'language': {'description': 'language of the country', 'title': 'Language', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'title': 'Capital', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'title': 'Country', 'type': 'object'}}, 'ls_structured_output_format': {'schema': {'description': 'Information about a country ', 'properties': {'name': {'description': 'name of the country', 'title': 'Name', 'type': 'string'}, 'language': {'description': 'language of the country', 'title': 'Language', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'title': 'Capital', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'title':

In [8]:
structured_llm.invoke("Tell me about Spain")

Country(name='Spain', language='Spanish', capital='Madrid')

### For No Validations

In [9]:
!pip install --upgrade typing-extensions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
ibis-framework 9.5.0 requires toolz<1,>=0.11, but you have toolz 1.0.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Li

In [10]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

In [11]:
 class Joke(BaseModel):
    """ Joke to tell the user """

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "how much funny the joke is, from 1 to 10 "]


In [12]:
structured_llm = llm.with_structured_output(Joke)

In [13]:
structured_llm.invoke("Tell me a joke about .cats")

Joke(setup='Why did the cat join a band', punchline='becuase it wnted to be a purr-cussionist', rating=8)